<h1>Scrap Computer Products</h1>
First thing with only scrapy project is all about running these commands,
<ul>
<li>scrapy startproject folder_name : make a folder with scrapy struture project, then cd to the project</li>
<li>python -m venv venv : create a venv file</li>
<li>.\venv\Scripts\Activate.ps1 : to activate vene</li>
<li>scrapy genspider spider_name url_to_scrap : create spider with default parse func</li>
</ul>

And the result as default is...

In [ ]:
import scrapy


class ScrapProductsSpider(scrapy.Spider):
    name = "scrap_products"
    allowed_domains = ["www.goldonecomputer.com"]
    start_urls = ["https://www.goldonecomputer.com/"]

    def parse(self, response):
        links = response.xpath('')

<h3>First : Scrap all categories links</h3>

In [ ]:
def parse(self, response):
        # Scrap all links in categories
        links = response.xpath('//*[@id="res-menu"]/ul/li/a')
        print(links)

        for link in links:
            category = link.css('::text').get()
            url = link.xpath('@href').get()

            # Validate the unwanted and none response
            unwanted_titles = ['home', 'contact', 'about']

            if not category or category.lower() in unwanted_titles or not url:
                print("Skippy none title and url...")
                continue

            # Response for categories.json 
            yield {
                    'category' : category,
                    'url' : url
                }

<h3>Second : Follow Category Links and Scrap all product detail links

In [ ]:
import scrapy


class ScrapProductsSpider(scrapy.Spider):
    name = "scrap_products"
    allowed_domains = ["www.goldonecomputer.com"]
    start_urls = ["https://www.goldonecomputer.com/"]

    def parse(self, response):
        # Scrap all links in categories
        links = response.xpath('//*[@id="res-menu"]/ul/li/a')
        print(links)

        for link in links:
            category = link.css('::text').get()
            url = link.xpath('@href').get()

            # Validate the unwanted and none response
            unwanted_titles = ['home', 'contact', 'about']

            if not category or category.lower() in unwanted_titles or not url:
                print("Skippy none title and url...")
                continue


            yield scrapy.Request(
                url = url,
                callback = self.parse_categories,
                meta = {'category' : category}
            )

    def parse_categories(self, response):
        category = response.meta['category']
        detail_links = response.xpath('//div[@id="content"]//div[contains(@class, "product-block product-thumb")]//a')

        products = {}
        for link in detail_links:
            product = link.css('::text').get()
            product_link = link.xpath('@href').get()

            if not product and product_link:
                print("Skipping the none product and product link...")
                continue    

            products[product] = product_link   

        yield {
                category : products
            } 
                
            


or

In [ ]:
import scrapy
import json

class ScrapProductLinksSpider(scrapy.Spider):
    name = "scrap_product_links"
    allowed_domains = ["www.goldonecomputer.com"]
    start_urls = ["https://www.goldonecomputer.com/"]

    def request_each_links(self):
        with open('categories_links.json', 'r') as f:
            urls = json.laod(f)
        for item in urls:
            yield scrapy.Request(url=item['url'], callback=self.parse)

    def parse(self, response):
        detail_links = response.xpath('//div[@id="content"]//div[contains(@class, "product-block product-thumb")]//a')

        for link in detail_links:
            product = link.css('::text').get()
            product_link = link.xpath('@href').get()

            if not product and product_link:
                print("Skipping the none product and product link...")
                continue
            
            yield {
                'product' : product,
                'link' : product_link
            }



<h3>The Last Updated : </h3>
<ul>
<li><b>def parse(self, response):</b> use for scrap all the category links, after get links loop it. Each link, use scrapy.Request to call the parse_category_page to work</li>
<li><b>def parse_category_page(self, response):</b> use for scrap all the product links after goes to each category. Then loop for each link call the parse_product_detail to work. But for the pagination we handle with response.follow to follow next page</li>
<li><b>def parse_product_detail(self, response):</b> use for scrap all detail data of the product from the gaven link. Use item = ScrapComProductsItem to​ build the appearence of the data with item.py and pipeline.py</li>
</ul>

In [ ]:
import scrapy
from scrap_com_products.items import ScrapComProductsItem


class ScrapProductsSpider(scrapy.Spider):
    name = "scrap_products"
    allowed_domains = ["www.goldonecomputer.com"]
    start_urls = ["https://www.goldonecomputer.com/"]

    def parse(self, response):
        
        # Get category links
        links = response.xpath('//*[@id="res-menu"]/ul/li/a')
        # print(f'Found {len(links)} of total links')

        for link in links:
            category = link.xpath('string(.)').get(default='No Category').strip()
            url = link.xpath('@href').get()

            # Skip unwanted categories
            unwanted = ['home', 'contact', 'about']
            
            if not category or not url or category.lower() in unwanted:
                print(f"Skipping none category or url : {category}")
                continue

            # print(f"=====> Processing category: {category}")
            
            yield scrapy.Request(
                url=url,
                callback=self.parse_category_page,
                meta={'category': category}
            )

    def parse_category_page(self, response):
        category = response.meta['category']
        
        product_links = []
        
        detail_links = response.xpath('//div[contains(@class, "product-thumb")]//h4/a/@href').getall()
        if detail_links:
            product_links = detail_links
            # print(f"Found {len(detail_links)} product detail_links")

        for product_url in product_links:
            yield scrapy.Request(
                url=product_url,
                callback=self.parse_product_detail,
                meta={'category': category}
            )

        next_page = response.xpath('//ul[@class="pagination"]//a[contains(text(), ">")]/@href').get()
        if next_page:
            yield response.follow(
                next_page,
                callback=self.parse_category_page,
                meta={'category': category}
            )

    def parse_product_detail(self, response):
        category = response.meta['category']
        title = response.xpath('//div[@id="content"]//h3/text()').get(default='No Title').strip()
        brand = response.xpath('//ul[@class="list-unstyled"]/li[span[text()="Brand:"]]/a/text()').get(default='No Brand').strip()
        code = response.xpath('//ul[@class="list-unstyled"]/li[span[text()="Product Code:"]]/text()').get(default='No Code').replace('Product Code:', '').strip()
        price = response.xpath('//ul[@class="list-unstyled price"]/li/h3/text()').get(default='No Price').strip()
        image_link = response.xpath('//a[@class="thumbnail"]/@href').get()
        if not image_link:
            image_link = response.xpath('//*[@id="content"]/div[1]/div[1]/div/div/div//img/@src').get()
        review = response.xpath('//a[contains(@class, "review-count")]/text()').get(default='No Review').strip()
            
             # products = {}
        # for url in product_links:
        #     products = {
        #         'title': title,
        #         'brand': brand,
        #         'code': code,
        #         'price': price,
        #         'image_link': image_link,
        #         'review': review,
        #         'url' : url
        #     }
        
        # yield{
        #     category : products
        # }
            
            
        # Create the item
        item = ScrapComProductsItem(
            category=category,
            product_data={
                'category' : category,
                'title': title,
                'brand': brand,
                'code': code,
                'price': price,
                'image_link': image_link,
                'review': review,
            }
        )
        
        yield item